In [103]:
import os
import glob
import re
import pandas as pd
from dodfminer.extract.polished.acts.aposentadoria import Retirements
import numpy as np
from itertools import zip_longest

In [79]:
# ls data/dodfs_flat/

In [80]:
def get_reverse_date(s):
    date = re.search( r'((\d+)[-](\d+)[-](\d+))', s)
    date = date.group()
    date = '-'.join(date.split('-')[::-1])
    return date

def get_num(s):
    return int(re.search('(\d+)', s).group())

In [81]:
df = pd.DataFrame()

## Quantos DODFs baixados?

In [82]:
dodfs_txt = glob.glob('data/dodfs_flat/*.txt')
df['file'] = [i.split('/')[-1] for i in dodfs_txt]
df['date'] = [get_reverse_date(s) for s in df.file]
# df['date'] = pd.to_datetime(df.date)
df['num'] = [get_num(s) for s in df.file]
df.drop('file', axis=1, inplace=True)
len(dodfs_txt)

595

## Quantos tiveram aposentadoria detectada?

In [83]:
%%time
dodfs_ext = [Retirements(i, 'regex') for i in dodfs_txt]
df['has_act'] = [not i.data_frame.empty for i in dodfs_ext] 
dodfs_ext = [i for i in dodfs_ext if not i.data_frame.empty]
print(
    "TOTAL DE ARQUIVOS COM APOSENTADORIAS:",
    df[~df.has_act].shape[0]
)

TOTAL DE ARQUIVOS COM APOSENTADORIAS: 368
CPU times: user 11.9 s, sys: 166 ms, total: 12.1 s
Wall time: 12.1 s


In [84]:
douro = pd.read_csv('data/aposentadoria-ouro/aposentaria-ouro-®.csv',
            sep='®', engine='python', index_col=False,)
douro.shape

(5516, 19)

In [85]:
douro.groupby(['DATA_DODF', 'NUM_DODF']).count().shape

(227, 17)

In [86]:
gp_ouro = douro.groupby(
    ['DATA_DODF', 'NUM_DODF']).count().iloc[:, 0:0].index
gp_miner = df[df.has_act].groupby(
    ['date', 'num']).count().iloc[:, 0:0].index

In [87]:
len(gp_miner), len(gp_ouro)

(226, 227)

In [88]:
sminer = set(gp_miner)
souro = set(gp_ouro)

In [89]:
MATTERS = ['DATA_DODF', 'NUM_DODF']
matters = ['date', 'num']
df_dtnum = df[df.has_act][matters].groupby(matters).count()
df_dtnum.reset_index(inplace=True)

douro_dtnum = douro[MATTERS].groupby(MATTERS).count()
douro_dtnum.reset_index(inplace=True)
douro_dtnum.rename({'DATA_DODF':'date', 'NUM_DODF':'num'},
                   inplace=True, axis=1)

In [90]:
souro_date = set(douro_dtnum.date.astype(str))
sdf_date = set(df_dtnum.date.astype(str))

In [105]:
duplis = list(zip_longest(douro_dtnum.values, df_dtnum.values))
eq = [i for i in duplis if np.array_equal(*i)]
neq = [i for i in duplis if not np.array_equal(*i)]
len(eq), len(neq)

(28, 199)

In [109]:
neq

[(array(['2018-04-19', 75], dtype=object),
  array(['2018-04-20', 76], dtype=object)),
 (array(['2018-04-20', 76], dtype=object),
  array(['2018-04-23', 77], dtype=object)),
 (array(['2018-04-23', 77], dtype=object),
  array(['2018-04-25', 79], dtype=object)),
 (array(['2018-04-25', 79], dtype=object),
  array(['2018-04-26', 80], dtype=object)),
 (array(['2018-04-26', 80], dtype=object),
  array(['2018-04-30', 82], dtype=object)),
 (array(['2018-04-30', 82], dtype=object),
  array(['2018-05-02', 83], dtype=object)),
 (array(['2018-05-02', 83], dtype=object),
  array(['2018-05-03', 84], dtype=object)),
 (array(['2018-05-03', 84], dtype=object),
  array(['2018-05-04', 85], dtype=object)),
 (array(['2018-05-04', 85], dtype=object),
  array(['2018-05-07', 86], dtype=object)),
 (array(['2018-05-07', 86], dtype=object),
  array(['2018-05-08', 87], dtype=object)),
 (array(['2018-05-08', 87], dtype=object),
  array(['2018-05-09', 88], dtype=object)),
 (array(['2018-05-09', 88], dtype=object),


In [107]:
np.array_equal( duplis[0] , duplis[1])

False

In [108]:
np.array_equal(duplis[0][0], duplis[0][1])

True

In [54]:
df_dtnum.shape, douro_dtnum.shape

((226, 2), (227, 2))